Pass or fail based on critieria 
final grade prediction for values that do not have a final grade 
a knn (idk which one yet)

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pylab as plt
from dmba import classificationSummary

In [11]:
student_performace = pd.read_csv('student_performance_updated_1000.csv')


In [ ]:
#clean data
student_performace.columns = [s.strip().replace(' ','_') for s in student_performace.columns]
print(student_performace.describe())
reduced_performace = student_performace.dropna()
reduced_performace = reduced_performace[
    (reduced_performace['StudyHoursPerWeek'] > 0) |
    (reduced_performace['AttendanceRate'] < 100) |
    (reduced_performace['StudyHoursPerWeek'] < (24 * 7)) |
    (reduced_performace['FinalGrade'] <= 100)
]
x = reduced_performace.drop(columns=['FinalGrade'])
y = reduced_performace['FinalGrade']
reduced_performace.head()

,StudentID,Name,Gender,AttendanceRate,StudyHoursPerWeek,PreviousGrade,ExtracurricularActivities,ParentalSupport,FinalGrade,Study_Hours,Attendance_(%),Online_Classes_Taken
0,1.0,John,Male,85.0,15.0,78.0,1.0,High,80.0,4.8,59.0,False
1,2.0,Sarah,Female,90.0,20.0,85.0,2.0,Medium,87.0,2.2,70.0,True
2,3.0,Alex,Male,78.0,10.0,65.0,0.0,Low,68.0,4.6,92.0,False
3,4.0,Michael,Male,92.0,25.0,90.0,3.0,High,92.0,2.9,96.0,False
6,7.0,Daniel,Male,70.0,8.0,60.0,0.0,Low,62.0,4.5,96.0,False


In [ ]:
#descriptive statistics


         StudentID  AttendanceRate  StudyHoursPerWeek  PreviousGrade  \
count   645.000000      645.000000         645.000000     645.000000   
mean   5419.272868       85.378295          17.682171      77.931783   
std    2682.454600        7.282423           6.355641      10.060917   
min       1.000000       70.000000           8.000000      60.000000   
25%    3044.000000       82.000000          12.000000      70.000000   
50%    5425.000000       85.000000          18.000000      82.000000   
75%    7812.000000       91.000000          22.000000      86.000000   
max    9991.000000       95.000000          30.000000      90.000000   

       ExtracurricularActivities  FinalGrade  Study_Hours  Attendance_(%)  
count                 645.000000  645.000000   645.000000      645.000000  
mean                    1.536434   80.122481     2.426512       77.052713  
std                     1.041375    9.609561     1.578680       18.245148  
min                     0.000000   62.000000   

In [14]:
#predicting a students final grade based on other factors
print(reduced_performace.columns)
reduced_performace1= pd.get_dummies(reduced_performace,drop_first=True,dtype=int,columns=['ParentalSupport'])
print(reduced_performace1.columns)

Index(['StudentID', 'Name', 'Gender', 'AttendanceRate', 'StudyHoursPerWeek',
       'PreviousGrade', 'ExtracurricularActivities', 'ParentalSupport',
       'FinalGrade', 'Study_Hours', 'Attendance_(%)', 'Online_Classes_Taken'],
      dtype='object')
Index(['StudentID', 'Name', 'Gender', 'AttendanceRate', 'StudyHoursPerWeek',
       'PreviousGrade', 'ExtracurricularActivities', 'FinalGrade',
       'Study_Hours', 'Attendance_(%)', 'Online_Classes_Taken',
       'ParentalSupport_Low', 'ParentalSupport_Medium'],
      dtype='object')
